In [55]:
import labrad
from labrad.units import WithUnit
import time
from math import log
import matplotlib.pyplot as plt

In [56]:
import labrad
scope = labrad.connection().dsox3034a
scope.connect((0x0957, 0x17a6, 'MY54100486')) #include S/N if more than one device
scope.setAcquireType('hres')
scope.setWaveformPoints(100)

In [57]:
cxn = labrad.connect()
pulser = cxn.pulser

In [47]:
def pON(n):
    return pulser.output('DDS_' + str(n), True)

def pOFF(n):
    return pulser.output('DDS_' + str(n), False)

In [5]:
pulser.amplitude('DDS_0', WithUnit(10, 'dBm'))
pulser.frequency('DDS_0', WithUnit(50, 'MHz'))

Value(50.0, 'MHz')

In [483]:
#delays determined by characterizing equipment
global_delay = 0.375
abs_phase = 0   
f_delay = 0.2
p_delay = 0.2      #0.218 + 0.14
a_delay = 0.2
total_delay = f_delay + p_delay + a_delay
f_delay_u = WithUnit(f_delay, 'us')
p_delay_u = WithUnit(p_delay, 'us')
a_delay_u = WithUnit(a_delay, 'us')
off_amp = WithUnit(-63, 'dBm')

def split(DDS_sequence): 
    off_amp = WithUnit(-63, 'dB')
    new_sequence = []
    
    for dds in DDS_sequence:
        start = dds[1][dds[1].units] #us
        duration = dds[2][dds[2].units] #us
        p_start_u = WithUnit(start - f_delay - p_delay - a_delay, 'us')
        f_start_u = WithUnit(start - f_delay - a_delay, 'us')
        a_start_u = WithUnit(start - a_delay, 'us')
        dds_phase = [(dds[0], p_start_u, p_delay_u, dds[3], WithUnit(-63, 'dBm'), dds[5], dds[6], dds[7])]
        dds_freq = [(dds[0], f_start_u, f_delay_u, dds[3], WithUnit(-63, 'dBm'), dds[5], dds[6], dds[7])]
        dds_amp = [(dds[0], a_start_u, WithUnit(a_delay + duration, 'us'), dds[3], dds[4], dds[5], dds[6], dds[7])]
        #dds_off = [(dds[0], a_start_u, WithUnit(duration, 'us'), dds[3], WithUnit(-63, 'dBm'), dds[5], dds[6], dds[7])]
        new_sequence += dds_phase
        new_sequence += dds_freq
        new_sequence += dds_amp
    print new_sequence
    return new_sequence

def check(DDS_sequence): 
    for n in range(0, len(DDS_sequence)-1):
        curr_end = DDS_sequence[n][1][DDS_sequence[n][1].units] + DDS_sequence[n][2][DDS_sequence[n][2].units]
        next_start = DDS_sequence[n+1][1][DDS_sequence[n+1][1].units]
        if (next_start - curr_end <= total_delay): 
            raise ValueException("Time between pulse ", n, " and pulse ", n+1, " is too short by ", abs(next_start - curr_end), ".")      

def phase_coherent(DDS_sequence): 
    new_sequence= []
    frequencies_times = {}
    phases = [] #stores relative phase shifts in sequential order
    abs_phase = 0   
    last_end = 0
    last_freq = 0
    for dds in DDS_sequence: 
        frequency = dds[3][dds[3].units]
        start = dds[1][dds[1].units]
        duration = dds[2][dds[2].units]
        end = start + duration
        print frequency, start, duration, end 
        abs_phase = compute_phase(abs_phase, start-last_end, last_freq)%360
        if frequency not in frequencies_times: 
            print "last_end: ", last_end
            print "abs_phase 1: ", abs_phase
            phases.append(0) #no phase shift needed for first pulse of frequency
            abs_phase = compute_phase(abs_phase, duration, frequency)
            print "abs_phase 2: ", abs_phase
            frequencies_times[frequency] = (frequency, end, abs_phase)
            last_end = end
            last_freq = frequency
        else:
            print "last_end:", last_end
            dt = start - frequencies_times[frequency][1] #computes time difference between start of pulse and end of last pulse of same frequency
            print "dt: ", dt #why is this 19.4 - because the code doesn't consider the zero a
            print "abs_phase 1: ", abs_phase
            #relative_phase = abs_phase - frequencies_times[frequency][2] #calculates relative phase shift from current pulse by finding difference between absolute phase at end of last puls of same frequency and absoluate phase at start of current pulse
            last_phase = compute_phase(abs_phase, dt, frequency)
            relative_phase = rel_phase(abs_phase, frequencies_times[frequency][2])
            print "relative_phase: ", relative_phase
            phases.append(relative_phase)
            abs_phase = (relative_phase + compute_phase(abs_phase, duration, frequency))%360
            print "abs_phase 2: ", abs_phase
            frequencies_times[frequency] = (frequency, end, abs_phase)
            last_end = end
            frequencies_times[frequency] = (frequency, last_end, abs_phase)
            last_freq = frequency
    #modify each entry in input to correct phase     
    for n in range(0, len(DDS_sequence)):
        new_dds = [(DDS_sequence[n][0], DDS_sequence[n][1], DDS_sequence[n][2], DDS_sequence[n][3], DDS_sequence[n][4], WithUnit(phases[n], 'deg'), DDS_sequence[n][6], DDS_sequence[n][7])]
        print phases[n]
        new_sequence+=(new_dds)
    return new_sequence     


def phase_coherent_1(DDS_sequence): 
    raw_pulses = [][]
    new_pulses = [][]
    for n in range(len(DDS_sequence)):
        dds = DDS_sequence[n]
        frequency = dds[3][dds[3].units]
        start = dds[1][dds[1].units]
        duration = dds[2][dds[2].units]
        end = start + duration
        raw_pulses[n][0] = frequency
        raw_pulses[n][1] = start
        raw_pulses[n][2] = duration 
        raw_pulses[n][3] = end
    new_pulses.append(raw_pulses[0])
    for m in range(1, len(raw_pulses)): 
        curr = raw_pulses[m]
        last = raw_pulses[m-1]
        gap = curr[1] - last[3]
        if gap != 0: 
            zero_pulse = [last[0], last[3], gap, curr[1]]
            new_pulses.append[zero_pulse]
        new_pulses.append(curr)
    
    

def rel_phase(p1, p2): 
    print "p1: ", p1
    print "p2: ", p2
    diff = p1-p2
    if diff < 0: 
        return (diff+360)%360
    else: 
        return diff%360

def compute_phase(initial_phase, dt, frequency): 
    #print "COMPUTE PHASE"
    phase = (dt*frequency*360)%360
    #print initial_phase, dt, frequency, phase
    return initial_phase + phase
                            
def process_sequence(dds_sequence): 
    return phase_coherent(split(dds_sequence))


In [480]:
compute_phase(0, 2, 1)

0

In [481]:
amp = WithUnit(8, 'dBm')
amp1 = WithUnit(15, 'dBm')
phase = WithUnit(0,'deg')
flip_phase = WithUnit(180,'deg')
amp_ramp_rate = WithUnit(0,'dB')
freq_ramp_rate = WithUnit(0,'MHz')

#min gap is 2 us

DDS_1 = \
[('DDS_1', WithUnit(10, 'us'), WithUnit(10.00,  'us'), WithUnit(100.0,   'MHz'), amp, phase, freq_ramp_rate, amp_ramp_rate),
 ('DDS_1', WithUnit(40, 'us'), WithUnit(10.00,  'us'), WithUnit(50.0,   'MHz'), amp, phase, freq_ramp_rate, amp_ramp_rate),
 ('DDS_1', WithUnit(70,'us'), WithUnit(10.00,  'us'), WithUnit(100.0,   'MHz'), amp, phase, freq_ramp_rate, amp_ramp_rate),
]

DDS_1_ttl = [(10,10), (40,10), (70,10)]

In [485]:
### DEFINE DDS SEQUENCE ###
t3 = [
#('DDS_1', WithUnit(5,'us'), WithUnit(110.00,  'us'), WithUnit(80.0,   'MHz'), amp, phase, freq_ramp_rate, amp_ramp_rate),
('DDS_2', WithUnit(10,'us'), WithUnit(10.00,  'us'), WithUnit(1.0,   'MHz'), amp1, phase, freq_ramp_rate, amp_ramp_rate),
('DDS_2', WithUnit(40,'us'), WithUnit(10.00,  'us'), WithUnit(1.0,   'MHz'), amp1, phase, freq_ramp_rate, amp_ramp_rate),
('DDS_2', WithUnit(70,'us'), WithUnit(10.00,  'us'), WithUnit(1.0,   'MHz'), amp1, phase, freq_ramp_rate, amp_ramp_rate),
('DDS_2', WithUnit(100,'us'), WithUnit(10.00,  'us'), WithUnit(1.0,   'MHz'), amp1, phase, freq_ramp_rate, amp_ramp_rate),
]

### DEFINE TTL SEQUENCE ###
t3_switch_ttl = [(10,10), (40,10), (70,10), (100, 10)]

### RUN PROCEDURE ###
print "Starting sequence..."

scope.single()
time.sleep(1)
pulser.amplitude('DDS_1', amp1)
pulser.frequency('DDS_1', WithUnit(80.0,   'MHz'))
pulser.amplitude('DDS_2',WithUnit(-63,'dBm'))
pulser.new_sequence()
        
for ttl in t3_switch_ttl: 
    pulser.add_ttl_pulse('ttl_3',WithUnit(ttl[0],'us'),WithUnit(ttl[1],'us'))
        
pulser.add_dds_pulses(process_sequence(t3))
pulser.program_sequence()
pulser.start_number(1)
pulser.wait_sequence_done()
pulser.stop_sequence()
pulser.amplitude('DDS_1',WithUnit(-63,'dBm'))
pulser.amplitude('DDS_2',WithUnit(-63,'dBm'))
time.sleep(1)
print "Sequence done."

Starting sequence...
[('DDS_2', Value(9.400000000000002, 'us'), Value(0.2, 'us'), Value(1.0, 'MHz'), Value(-63.0, 'dBm'), Value(0.0, 'deg'), Value(0.0, 'MHz'), Value(0.0, 'dB')), ('DDS_2', Value(9.600000000000001, 'us'), Value(0.2, 'us'), Value(1.0, 'MHz'), Value(-63.0, 'dBm'), Value(0.0, 'deg'), Value(0.0, 'MHz'), Value(0.0, 'dB')), ('DDS_2', Value(9.8, 'us'), Value(10.2, 'us'), Value(1.0, 'MHz'), Value(15.0, 'dBm'), Value(0.0, 'deg'), Value(0.0, 'MHz'), Value(0.0, 'dB')), ('DDS_2', Value(39.39999999999999, 'us'), Value(0.2, 'us'), Value(1.0, 'MHz'), Value(-63.0, 'dBm'), Value(0.0, 'deg'), Value(0.0, 'MHz'), Value(0.0, 'dB')), ('DDS_2', Value(39.599999999999994, 'us'), Value(0.2, 'us'), Value(1.0, 'MHz'), Value(-63.0, 'dBm'), Value(0.0, 'deg'), Value(0.0, 'MHz'), Value(0.0, 'dB')), ('DDS_2', Value(39.8, 'us'), Value(10.2, 'us'), Value(1.0, 'MHz'), Value(15.0, 'dBm'), Value(0.0, 'deg'), Value(0.0, 'MHz'), Value(0.0, 'dB')), ('DDS_2', Value(69.39999999999999, 'us'), Value(0.2, 'us'), Va

In [255]:
scope.single()

In [248]:
### RUN PROCEDURE ###
print "Starting sequence..."

scope.single()
time.sleep(1)
pulser.amplitude('DDS_1',WithUnit(-63,'dBm'))
pulser.amplitude('DDS_2',WithUnit(-63,'dBm'))
pulser.new_sequence()
        
for ttl in t3_switch_ttl: 
    pulser.add_ttl_pulse('ttl_3',WithUnit(ttl[0],'us'),WithUnit(ttl[1],'us'))
        
pulser.add_dds_pulses(t3)
pulser.program_sequence()
pulser.start_number(1)
pulser.wait_sequence_done()
pulser.stop_sequence()
pulser.amplitude('DDS_1',WithUnit(-63,'dBm'))
pulser.amplitude('DDS_2',WithUnit(-63,'dBm'))
time.sleep(1)
print "Sequence done."

Starting sequence...
Sequence done.


In [243]:
pulser.amplitude('DDS_1',WithUnit(5,'dBm'))
pulser.amplitude('DDS_2',WithUnit(10,'dBm'))
pulser.frequency('DDS_1',WithUnit(60,'MHz'))
pulser.frequency('DDS_2',WithUnit(60,'MHz'))

Value(60.0, 'MHz')

In [318]:
pulser

LabRAD Server: Pulser (ID=4)




Settings:
    add_dds_pulses
    add_ttl_pulse
    add_ttl_pulses
    amplitude
    clear_dds_lock
    complete_infinite_iteration
    debug
    echo
    extend_sequence_length
    frequency
    get_channels
    get_collection_mode
    get_collection_time
    get_dds_amplitude_range
    get_dds_channels
    get_dds_frequency_range
    get_line_trigger_limits
    get_pmt_counts
    get_readout_counts
    get_secondary_pmt_counts
    get_state
    get_timetag_resolution
    get_timetags
    human_readable_dds
    human_readable_ttl
    internal_advance_dds
    internal_reset_dds
    line_trigger_duration
    line_trigger_state
    new_sequence
    output
    program_sequence
    reinitialize_dds
    repeatitions_completed
    reset_fifo_normal
    reset_readout_counts
    reset_timetags
    set_collection_time
    set_mode
    signal__log
    signal__new_dds_parameter
    signal__new_line_trigger_parameter
    signal__switch_toggled
    start_infinite
   